In [ ]:
!pip install transformers datasets bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (41

In [ ]:
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from datasets import load_dataset
import logging
import accelerate

In [ ]:
# Load the dataset
scitldr = load_dataset("allenai/scitldr")

# Load the tokenizer for preprocessing
model_path = "google/gemma-2b"  # Replace with the actual model path or name
tokenizer = AutoTokenizer.from_pretrained(model_path)

prefix = "summarize: "

def preprocess_function(examples):
    try:
        # Convert lists to strings
        source_str = " ".join(examples["source"])
        target_str = " ".join(examples["target"])

        # Add prefix to source
        inputs = prefix + source_str

        # Tokenize inputs
        model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

        # Tokenize labels
        labels = tokenizer(target_str, max_length=512, truncation=True, padding="max_length")

        # Add labels to model_inputs
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    except Exception as e:
        logging.error(f"Error in preprocessing function: {e}")
        return None

try:
    tokenized_scitldr = scitldr.map(preprocess_function, batched=True, remove_columns=scitldr["train"].column_names)
except Exception as e:
    logging.error(f"Error mapping dataset: {e}")

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=None)


The repository for allenai/scitldr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/scitldr.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/1992 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/618 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/619 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

...

ERROR:root:Error in preprocessing function: sequence item 0: expected str instance, list found
ERROR:root:Error in preprocessing function: sequence item 0: expected str instance, list found


Map:   0%|          | 0/618 [00:00<?, ? examples/s]

ERROR:root:Error in preprocessing function: sequence item 0: expected str instance, list found


Map:   0%|          | 0/619 [00:00<?, ? examples/s]

ERROR:root:Error in preprocessing function: sequence item 0: expected str instance, list found


In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Load the model with the correct device map and low CPU memory usage
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")



config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
def formatting_prompts_func(input_text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True, padding="max_length")
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Move tensors to the same device as the model

    # Generate summary
    summary_ids = model.generate(inputs["input_ids"], max_length=1024, num_beams=4, length_penalty=2.0, early_stopping=True, max_new_tokens=500)

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Construct the formatted output
    output_text = f"Input Text:\n{input_text}\n\nGenerated Summary:\n{summary}"

    return output_text

# Test the function with a sample input text
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use CUDA if available
model.to(device)  # Move model to the device
input_text = "Cricket, a game revered as much for its strategic depth as for its thrilling moments, captivates fans worldwide. The clash between bat and ball, the ebb and flow of the game, and the intense rivalries between teams make every match a spectacle. From the graceful cover drive to the fierce bouncer, cricket is a sport that blends skill, tactics, and sheer passion, drawing millions to stadiums and screens alike."
formatted_output = formatting_prompts_func(input_text)
print(formatted_output)



Both `max_new_tokens` (=500) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Input Text:
Cricket, a game revered as much for its strategic depth as for its thrilling moments, captivates fans worldwide. The clash between bat and ball, the ebb and flow of the game, and the intense rivalries between teams make every match a spectacle. From the graceful cover drive to the fierce bouncer, cricket is a sport that blends skill, tactics, and sheer passion, drawing millions to stadiums and screens alike.

Generated Summary:
Cricket, a game revered as much for its strategic depth as for its thrilling moments, captivates fans worldwide. The clash between bat and ball, the ebb and flow of the game, and the intense rivalries between teams make every match a spectacle. From the graceful cover drive to the fierce bouncer, cricket is a sport that blends skill, tactics, and sheer passion, drawing millions to stadiums and screens alike.

But what makes cricket so special? What is it about this game that has captured the hearts and minds of fans for centuries? In this article, we

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    num_train_epochs=3,  # Replace max_steps with num_train_epochs if you prefer
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    report_to='none',
    output_dir='logs',
    optim="adamw_hf",
    remove_unused_columns=False
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_scitldr['train'],
    data_collator=data_collator,
)




In [ ]:
trainer.train()

In [ ]:
!pip uninstall -y accelerate
!pip install accelerate==0.27.2


Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.6 MB/s eta 0:00:00


In [ ]:

print("Accelerate version:", accelerate.__version__)


Accelerate version: 0.21.0


In [ ]:
from datasets import load_dataset

# Load the dataset
scitldr = load_dataset("allenai/scitldr")

# Load the tokenizer for preprocessing
model_path = "google/gemma-2b"  # Replace with the actual model path or name
tokenizer = AutoTokenizer.from_pretrained(model_path)

prefix = "summarize: "

def preprocess_function(examples):
    try:
        # Convert lists to strings
        source_str = " ".join(examples["source"])
        target_str = " ".join(examples["target"])

        # Add prefix to source
        inputs = prefix + source_str

        # Tokenize inputs
        model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")

        # Tokenize labels
        labels = tokenizer(target_str, max_length=512, truncation=True, padding="max_length")

        # Add labels to model_inputs
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    except Exception as e:
        logging.error(f"Error in preprocessing function: {e}")
        return None

try:
    tokenized_scitldr = scitldr.map(preprocess_function, batched=True, remove_columns=scitldr["train"].column_names)
    # Inspect the first example to verify the structure
    print(tokenized_scitldr['train'][0])
except Exception as e:
    logging.error(f"Error mapping dataset: {e}")

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=None)

from transformers import AutoModelForSeq2SeqLM

# Load the model with the correct device map and low CPU memory usage
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b")

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    num_train_epochs=3,  # Replace max_steps with num_train_epochs if you prefer
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    report_to='none',
    output_dir='logs',
    optim="adamw_hf",
    remove_unused_columns=False
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_scitldr['train'],
    data_collator=data_collator,
)

trainer.train()


Map:   0%|          | 0/1992 [00:00<?, ? examples/s]

ERROR:root:Error in preprocessing function: sequence item 0: expected str instance, list found
ERROR:root:Error in preprocessing function: sequence item 0: expected str instance, list found


Map:   0%|          | 0/618 [00:00<?, ? examples/s]

ERROR:root:Error in preprocessing function: sequence item 0: expected str instance, list found


Map:   0%|          | 0/619 [00:00<?, ? examples/s]

ERROR:root:Error in preprocessing function: sequence item 0: expected str instance, list found


{'source': ['Due to the success of deep learning to solving a variety of challenging machine learning tasks, there is a rising interest in understanding loss functions for training neural networks from a theoretical aspect.', 'Particularly, the properties of critical points and the landscape around them are of importance to determine the convergence performance of optimization algorithms.', 'In this paper, we provide a necessary and sufficient characterization of the analytical forms for the critical points (as well as global minimizers) of the square loss functions for linear neural networks.', 'We show that the analytical forms of the critical points characterize the values of the corresponding loss functions as well as the necessary and sufficient conditions to achieve global minimum.', 'Furthermore, we exploit the analytical forms of the critical points to characterize the landscape properties for the loss functions of linear neural networks and shallow ReLU networks.', 'One partic

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB. GPU 